this notebook sees how to create a feedforward approximator for recurrent features extracted in `scripts/feature_extraction/yuanyuan_8k_a/maskcnn_polished_with_local_pcn/debug.ipynb`

In [1]:
# common libs
import numpy as np
import h5py

from sys import path
from os.path import join


from thesis_v2 import dir_dict

from torchnetjson.builder import build_net
from thesis_v2.training.training_aux import load_training_results

In [2]:
folder_to_check = 'scripts/feature_extraction/yuanyuan_8k_a/maskcnn_polished_with_local_pcn'
path.insert(0, join(dir_dict['root'], folder_to_check))
from certain_configs import get_all_model_params, global_vars

In [3]:
print(global_vars)

# utils
def get_layer_idx(friendly_name):
    return global_vars['augment_config']['module_names'].index(friendly_name)

{'feature_file_dir': '/my_data/thesis-yimeng-v2/results/features/maskcnn_polished_with_local_pcn/certain_configs', 'augment_config': {'module_names': ['bottomup', 'topdown', 'final'], 'name_mapping': {'moduledict.conv1.lambda_out': 'bottomup', 'moduledict.conv1.lambda_in': 'topdown', 'moduledict.final_act': 'final'}}}


In [4]:
# all model params
all_params_dict = get_all_model_params()

# same one as in `scripts/feature_extraction/yuanyuan_8k_a/maskcnn_polished_with_local_pcn/debug.ipynb`
model_to_check = 's_selegacy+in_sz50+out_ch16+num_l2+k_l19+k_p3+ptavg+bn_b_actTrue+bn_a_fcFalse+actrelu+p_c5+p_bypassFalse+p_n_actFalse+p_bn_pFalse+p_actTrue+p_bnTrue+p_biasTrue+sc0.01+sm0.000005+lmse+m_se0'

In [5]:
# function to extract input 'topdown.0' as well as all 'bottomup.*' except the last one.

def fetch_data(feature_file, grp_name):
    # let's collect first 10 images' responses for verification.
    slice_to_check = slice(0, 256)
    with h5py.File(feature_file, 'r') as f_feature:
        grp = f_feature[grp_name]
        num_bottom_up = len([x for x in grp if x.startswith(str(get_layer_idx('bottomup')) + '.')])
        assert num_bottom_up > 2
        
        pcn_in = grp[str(get_layer_idx('topdown')) + '.0'][slice_to_check]
        pcn_out_list = [grp[str(get_layer_idx('bottomup')) + f'.{x}'][slice_to_check] for x in range(num_bottom_up-1)]
    
    print((pcn_in.shape, pcn_in.mean(), pcn_in.std(), pcn_in.min(), pcn_in.max()))
    print([(x.shape, x.mean(), x.std(), x.min(), x.max()) for x in pcn_out_list])
    
    return {
        'in': pcn_in,
        'out_list': pcn_out_list,
    }

In [6]:
data_returned = fetch_data(join(global_vars['feature_file_dir'], model_to_check + '.hdf5'), 'X_test')

((256, 16, 42, 42), 0.35897052, 0.5706741, 0.0, 11.922759)
[((256, 16, 42, 42), 0.2553295, 0.42159, 0.0, 8.292999), ((256, 16, 42, 42), 0.27825132, 0.7672202, -4.185834, 13.933849), ((256, 16, 42, 42), 0.29259625, 1.0237633, -6.97853, 16.669125), ((256, 16, 42, 42), 0.3059992, 1.3043457, -9.800613, 19.26218), ((256, 16, 42, 42), 0.31785125, 1.650338, -13.161419, 21.148617), ((256, 16, 42, 42), 0.32922515, 2.0893815, -17.23216, 24.323668)]


In [7]:
# load the model
def load_model(key):
    result = load_training_results(key, return_model=False)
    # load twice, first time to get the model.
    model_ = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))['model']
    model_.cuda()
    model_.eval()
    return model_

model = load_model(all_params_dict[model_to_check]['key'])

In [8]:
# # the idea is, given idx1 and idx2, predict out_list[idx2] - out_list[idx1]  given (out_list[idx1]  and in).

from numpy.linalg import norm
from torch.backends import cudnn
import torch
cudnn.deterministic = True
cudnn.benchmark = False
def check_similarity(d1, d2):
    assert d1.shape == d2.shape
    norm_diff = norm(d1-d2)/norm(d2)
    print(norm_diff)
    print(abs(d1-d2).max())
    assert norm_diff < 1e-5

def debug_result(model_,in_,out1,idx_diff):
    assert idx_diff > 0
    out_now = out1
    with torch.no_grad():
        for _ in range(idx_diff):
            pred_now = model.moduledict['conv1'].forward_fb(
                torch.tensor(out_now).cuda(),
            ).cpu().numpy()
            out_now = model.moduledict['conv1'].forward_update(
                torch.tensor(out_now).cuda(),
                torch.tensor(in_).cuda(),
                torch.tensor(pred_now).cuda(),
            ).cpu().numpy()
    return out_now - out1

def check_result(model_, data_dict):
    num_out = len(data_dict['out_list'])
    
    for idx1 in range(num_out):
        for idx2 in range(idx1+1, num_out):
            print((idx1, idx2))
            result_ref = data_dict['out_list'][idx2] - data_dict['out_list'][idx1]
            print(result_ref.mean(), result_ref.std(), result_ref.min(), result_ref.max())
            result_debug = debug_result(model_,data_dict['in'],data_dict['out_list'][idx1],idx2-idx1)
            check_similarity(result_ref, result_debug)

# all ok.
check_result(model, data_returned)

(0, 1)
0.022921672 0.42046013 -4.185834 6.33709
0.0
0.0
(0, 2)
0.037266962 0.720724 -6.97853 10.422319
0.0
0.0
(0, 3)
0.05066977 1.0387783 -9.800613 13.015375
0.0
0.0
(0, 4)
0.06252189 1.4155142 -13.161419 15.393248
0.0
0.0
(0, 5)
0.073895715 1.8798158 -17.23216 17.793856
0.0
0.0
(1, 2)
0.014345304 0.33172116 -2.8825576 4.222127
0.0
0.0
(1, 3)
0.02774809 0.68103695 -6.032793 7.5598173
0.0
0.0
(1, 4)
0.0396002 1.0860796 -9.664015 9.994712
0.0
0.0
(1, 5)
0.050974116 1.5759 -14.066785 14.17896
0.0
0.0
(2, 3)
0.013402778 0.35958126 -3.1581278 3.3376904
0.0
0.0
(2, 4)
0.02525489 0.778254 -6.9140663 6.851712
0.0
0.0
(2, 5)
0.03662875 1.2837849 -11.316836 12.893453
0.0
0.0
(3, 4)
0.01185211 0.42521432 -3.7559385 4.227053
0.0
0.0
(3, 5)
0.023225963 0.9418687 -8.158709 10.268793
0.0
0.0
(4, 5)
0.011373837 0.52283007 -4.40277 6.144558
0.0
0.0


In [9]:
# now time to get a model to train it.
# simple stuff. conv + relu.
# maybe with BN.

# two kinds of models

# BN + conv + ReLU + BN
# conv + ReLU + BN
# I may want to constrain the first BN a bit,
# say, all in_ channels share the same scale and bias; same goes with out1 channels.

# some concerns: stats are different for `out_` at different iterations.
# but let's ignore it for now.